In [1]:
import numpy as np
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
import dimod
import importlib
import draw_grid
importlib.reload(draw_grid)

<module 'draw_grid' from 'C:\\Users\\tkemp\\dev\\quantum-experiments\\protein-folding\\draw_grid.py'>

In [10]:
import qubo_params
importlib.reload(qubo_params)

S = 0
L = 2

Q = qubo_params.get_Q(1, 3)

print(f'len(Q): {len(Q)}')
print(max(Q.values()))

1664129880.1836796
len(Q): 717
3.5


In [11]:
A = np.zeros((54, 54))
for ij in Q:
    A[ij] = Q[ij]
print(A[:8, :8])

[[-1.   3.5  2.   2.   2.   2.   2.   0. ]
 [ 0.  -1.   3.5  2.   2.   2.   0.   2. ]
 [ 0.   0.  -1.   3.5  2.   2.   0.   0. ]
 [ 0.   0.   0.  -1.   3.5  2.  -1.   0. ]
 [ 0.   0.   0.   0.  -1.   3.5  0.   0. ]
 [ 0.   0.   0.   0.   0.  -1.  -1.   0. ]
 [ 0.   0.   0.   0.   0.   0.  -1.   3.5]
 [ 0.   0.   0.   0.   0.   0.   0.  -1. ]]


In [12]:
relative_chain_strength = 0.4
chain_strength = max(Q.values()) * relative_chain_strength
print(f'chain strength: {chain_strength}')

n_runs = 100

chain strength: 1.4000000000000001


In [13]:
use_qpu = True
if use_qpu:
    bqm = dimod.BQM.from_qubo(Q)
    sampler = EmbeddingComposite(DWaveSampler(solver='Advantage2_prototype1.1'))
    response = sampler.sample(
      bqm,
      chain_strength=2.8,
      num_reads=490,
      annealing_time=100,
      label='Protein Folding (QUBO)',
    )
elif False:
    response = dimod.ExactSolver().sample_qubo(Q)
else:
    response = dimod.IdentitySampler().sample_qubo(Q, initial_states=[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])


2022-09-25 20:18:17,627 dwave.inspector.adapters ERROR MainThread Solver topology {'type': 'zephyr', 'shape': [4, 4]} not supported.


In [14]:
print("Energy Reached:", response.first.energy)
print(f"Number of records: {len(response.record)}")
solutions = response.lowest()
print(f'Number of solutions: {len(solutions)}')
print(solutions.slice(0, 10))
print(response.first)

Energy Reached: -8.0
Number of records: 421
Number of solutions: 2
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 ... 53 energy num_oc. ...
0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0 ...  0   -8.0       5 ...
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ...  0   -8.0       1 ...
['BINARY', 2 rows, 6 samples, 54 variables]
Sample(sample={0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 1, 22: 0, 23: 0, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0, 41: 0, 42: 0, 43: 1, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0}, energy=-8.0, num_occurrences=5, chain_break_fraction=0.037037037037037035)


In [15]:
response

SampleSet(rec.array([([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -8. , 5, 0.03703704),
           ([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -7. , 1, 0.03703704),
           ([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -7. , 1, 0.03703704),
           ([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -7. , 1, 0.03703704),
           ([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -7. , 1, 0.03703704),


In [17]:
for x in response.lowest():
    draw_grid.draw_grid_binary(np.array(list(x.values())), 3, 6)

[[5. 6. 0.]
 [4. 1. 0.]
 [3. 2. 0.]]
[[0. 0. 0.]
 [5. 4. 3.]
 [6. 1. 2.]]


In [18]:
embedding = response.info['embedding_context']['embedding']
print(f"Number of logical variables: {len(embedding.keys())}")
print(f"Number of physical qubits used in embedding: {sum(len(chain) for chain in embedding.values())}")

Number of logical variables: 54
Number of physical qubits used in embedding: 230


In [ ]:
if use_qpu:
    dwave.inspector.show(response)

2022-09-25 20:05:12,922 dwave.inspector.server ERROR Thread-5 Exception on /api/problems/4ddaa03c-3e81-411e-b1d6-1ca1226a915b [GET]
Traceback (most recent call last):
  File "c:\users\tkemp\ocean\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\tkemp\ocean\lib\site-packages\flask\app.py", line 1526, in full_dispatch_request
    return self.finalize_request(rv)
  File "c:\users\tkemp\ocean\lib\site-packages\flask\app.py", line 1545, in finalize_request
    response = self.make_response(rv)
  File "c:\users\tkemp\ocean\lib\site-packages\flask\app.py", line 1720, in make_response
    rv = jsonify(rv)
  File "c:\users\tkemp\ocean\lib\site-packages\flask\json\__init__.py", line 301, in jsonify
    f"{dumps(data, indent=indent, separators=separators)}\n",
  File "c:\users\tkemp\ocean\lib\site-packages\flask\json\__init__.py", line 131, in dumps
    return _json.dumps(obj, **kwargs)
  File "C:\Users\tkemp\AppData\Local\Progra

In [8]:
import sequences
importlib.reload(sequences)
S = sequences.structures
def z(structure, L):
    Ls = L**2
    N = len(structure)
    D = Ls * N
    x = np.zeros((D))
    for f, s in enumerate(structure):
        x[s * N + f] = 1
    return x
d = draw_grid.draw_grid_binary
d(z(S[0], 2), 2, 4)
d(z(S[1], 3), 3, 6)
d(z(S[2], 3), 3, 7)
d(z(S[3], 3), 3, 8)
d(z(S[4], 4), 4, 9)
d(z(S[5], 4), 4, 10)
d(z(S[6], 4), 4, 14)
print(list(z(S[0], 2)))

[[2. 3.]
 [1. 4.]]
[[2. 3. 0.]
 [1. 4. 0.]
 [6. 5. 0.]]
[[6. 5. 4.]
 [7. 2. 3.]
 [0. 1. 0.]]
[[2. 3. 4.]
 [1. 8. 5.]
 [0. 7. 6.]]
[[3. 4. 0. 0.]
 [2. 5. 6. 0.]
 [1. 8. 7. 0.]
 [0. 9. 0. 0.]]
[[ 0.  5.  6.  0.]
 [ 3.  4.  7.  8.]
 [ 2.  1. 10.  9.]
 [ 0.  0.  0.  0.]]
[[ 0.  9. 10. 11.]
 [ 7.  8. 13. 12.]
 [ 6.  5. 14.  1.]
 [ 0.  4.  3.  2.]]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [9]:
response2 = dimod.IdentitySampler().sample_qubo(Q, initial_states=[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])
print("Energy Reached:", response2.first.energy)
print(f"Number of records: {len(response2.record)}")
solutions = response2.lowest()
print(f'Number of solutions: {len(solutions)}')
print(response2.slice(0, 10))
print(response2.first)

Energy Reached: -5.0
Number of records: 1
Number of solutions: 1
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc.
0  0  1  0  0  0  0  1  0  1  0  0  0  0  0  0  1   -5.0       1
['BINARY', 1 rows, 1 samples, 16 variables]
Sample(sample={0: 0.0, 1: 1.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 1.0, 7: 0.0, 8: 1.0, 9: 0.0, 10: 0.0, 11: 0.0, 12: 0.0, 13: 0.0, 14: 0.0, 15: 1.0}, energy=-5.0, num_occurrences=1)
